# Home Task: Investigating Offer Selection Issues
# Context

Our product evaluates borrowers, and at the end of the process, they are either approved or declined.
If approved, they receive multiple flexible loan amount options.

# How the Process Works

1. Borrower Request

   - The process begins with the borrower (application) specifying a requested amount they would like to borrow.

2. Offer Generation & Evaluation

   - For every application, we simultaneously evaluate a standardized set of pre-defined loan amounts and terms (e.g., 5K at 36 or 60 terms, 10K at 36 or 60 terms, etc.).

   - If approved, on the borrower's side, a scroller allows them to choose an amount within the approved range (e.g., 5,000-30,000) with either a 36 or 60 term.

   - Our offer generation is specifically designed to enhance the user experience: by testing the entire range at once, we can instantly determine and present a flexible approved credit line.

3. How Offers Are Sent in the Response

   - For each combination of application ID, interest rate, and term (36/60), we consolidate the results into a range (minimum amount, maximum amount) instead of sending multiple offers with the same rate.

   - This approach makes the response both simpler and more flexible.

   - Each returned group includes:
     - minimum_amount: The lowest approved amount for that group
     - maximum_amount: The highest approved amount for that group
     - rate: The applicable interest rate
     - term: The applicable term (36 or 60 months)
     - From each group we assign "is_response" as True to the offer where loan_amount is equal to maximum_amount, the rest are set to False.

Note: For each borrower, there can be both approved and declined offers returned.

# Example - Approved Range:
minimum_amount: 10,000
maximum_amount: 25,000
term: 36
int_rate: 8.0

The borrower can then select any exact amount within this range using the scroller.

# Example - Declined Range:
minimum_amount: 30,000
maximum_amount: 35,000
term: 36
int_rate: NaN





# The Problem

Some users have reported that they cannot select their requested amount even though they can select higher or lower amounts within the range.

# The Task

Investigate the issue and provide the following:

1. Identify all rows in the dataset that can explain the user complaints.

2. Explain why this happens.

3. Count:
    - Total number of applications affected
    - Specifically, how many requested amounts cannot be issued due to this problem

4. Suggest a fix to prevent this from happening.

# What You Have

1. offers.parquet - Dataset representing production-run applications.



In [46]:
# Import required libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


## Validate (use as many cells you need)

In [59]:
# Load the datasets
offers = pd.read_parquet('offers.parquet')

print(f"Offers dataset: {offers.shape}")
print("Offers dataset columns:")
print(offers.columns.tolist())
offers.head(5)

Offers dataset: (462, 11)
Offers dataset columns:
['app_id', 'loan_amount', 'loan_term', 'requested_amount', 'state', 'is_approved', 'int_rate', 'credit_score', 'maximum_amount', 'minimum_amount', 'is_response']


,app_id,loan_amount,loan_term,requested_amount,state,is_approved,int_rate,credit_score,maximum_amount,minimum_amount,is_response
0,1000,5000,36,6500,VA,True,13.99,628,10000,5000,False
1,1000,8000,36,6500,VA,True,13.99,628,10000,5000,False
2,1000,10000,36,6500,VA,True,13.99,628,10000,5000,True
3,1000,15000,36,6500,VA,True,21.99,628,20000,15000,False
4,1000,20000,36,6500,VA,True,21.99,628,20000,15000,True


In [62]:
# 🔍 Start your investigation here

print("INVESTIGATION WORKSPACE")
print("=" * 50)

# TODO: Your analysis goes here

print("Start your pandas investigation here!")


INVESTIGATION WORKSPACE
Start your pandas investigation here!


In [49]:
# For each application (app_id), collect:
# - requested 
# - min_offer / max_offer 
# - options 
offers_summary = (
    offers.groupby("app_id")
    .agg(
        requested=("requested_amount", "first"),
        min_offer=("loan_amount", "min"),
        max_offer=("loan_amount", "max"),
        options=("loan_amount", lambda s: sorted(set(s)))
    )
    .reset_index()
)
print(f"offers_summary rows: {len(offers_summary)}, cols: {len(offers_summary.columns)}")
offers_summary.head(5)         


offers_summary rows: 33, cols: 5


,app_id,requested,min_offer,max_offer,options
0,1000,6500,5000,30000,"[5000, 8000, 10000, 15000, 20000, 25000, 30000]"
1,1001,6500,5000,30000,"[5000, 8000, 10000, 15000, 20000, 25000, 30000]"
2,1002,12000,5000,30000,"[5000, 8000, 10000, 15000, 20000, 25000, 30000]"
3,1003,12000,5000,30000,"[5000, 8000, 10000, 15000, 20000, 25000, 30000]"
4,1004,27500,5000,30000,"[5000, 8000, 10000, 15000, 20000, 25000, 30000]"


In [63]:
# Flag: requested inside [min_offer, max_offer]
in_range = (
    (offers_summary["requested"] >= offers_summary["min_offer"]) &
    (offers_summary["requested"] <= offers_summary["max_offer"])
)
print("in_range calcal: ")
print (in_range.value_counts())

in_range calcal: 
True     31
False     2
Name: count, dtype: int64


In [55]:
#check if requested amount is in the avalilable options
requested_in_options = []

for i, row in offers_summary.iterrows():
    is_in_options = str(row['requested']) in [str(n) for n in row['options']]
    requested_in_options.append(is_in_options)

#add the result as a new column
offers_summary['requested_in_options'] = requested_in_options
print("✅ 'requested_in_options' calculated:")
print(offers_summary['requested_in_options'].value_counts())

print("\n📋 Sample of requested vs options:")
print(offers_summary[["app_id", "requested", "options", "requested_in_options"]].head(5))

   

✅ 'requested_in_options' calculated:
requested_in_options
False    33
Name: count, dtype: int64

📋 Sample of requested vs options:
   app_id  requested                                          options  \
0    1000       6500  [5000, 8000, 10000, 15000, 20000, 25000, 30000]   
1    1001       6500  [5000, 8000, 10000, 15000, 20000, 25000, 30000]   
2    1002      12000  [5000, 8000, 10000, 15000, 20000, 25000, 30000]   
3    1003      12000  [5000, 8000, 10000, 15000, 20000, 25000, 30000]   
4    1004      27500  [5000, 8000, 10000, 15000, 20000, 25000, 30000]   

   requested_in_options  
0                 False  
1                 False  
2                 False  
3                 False  
4                 False  


In [64]:
# Combine both conditions to detect problem cases
offers_summary["problem"] = in_range & (~offers_summary["requested_in_options"])


In [ ]:
print("=" * 50)
print("FINNAL SUMMARY")
print("=" * 50)

final_summary = offers_summary.loc[
    offers_summary["problem"],
    ["app_id", "requested", "min_offer", "max_offer", "options", "requested_in_options", "problem"]
].copy()
print(final_summary.head(5).to_string(index=False))




FINNAL SUMMARY
 app_id  requested  min_offer  max_offer                                         options  requested_in_options  problem
   1000       6500       5000      30000 [5000, 8000, 10000, 15000, 20000, 25000, 30000]                 False     True
   1001       6500       5000      30000 [5000, 8000, 10000, 15000, 20000, 25000, 30000]                 False     True
   1002      12000       5000      30000 [5000, 8000, 10000, 15000, 20000, 25000, 30000]                 False     True
   1003      12000       5000      30000 [5000, 8000, 10000, 15000, 20000, 25000, 30000]                 False     True
   1004      27500       5000      30000 [5000, 8000, 10000, 15000, 20000, 25000, 30000]                 False     True


In [72]:
total_apps = len(offers_summary)
affected_apps = offers_summary["problem"].sum()

print("BUG STATISTICS")
print("=" * 60)
print(f"Total number of apps: {total_apps}")
print(f"Total number of apps affected: {affected_apps}")
print(f"Requested amounts that failed: {affected_apps}")
print(f"Clean applications (no bug, maybe or the requested amount is not in the min/max range): {total_apps - affected_apps}")
print("=" * 60)


BUG STATISTICS
Total number of apps: 33
Total number of apps affected: 31
Requested amounts that failed: 31
Clean applications (no bug, maybe or the requested amount is not in the min/max range): 2


In [ ]:
#The bug happen cause users can pick amounts that are inside the valid (min/max) range but not part of the offer list.
#To fix it the front-end and back-end should use the same logic for valid loan amounts.
#Also, add a quick validation step that checks if all amounts in the range are included in the options before showing them to the user.



# 🚨 Part 2: Business Rules Validation

## New Requirements

The development team has implemented new business rules that need validation:

### Credit Score Rules
- **Block all loans** with credit scores < 600
- **Block all loans** with credit scores ≥ 720

### State-Based Rate Restrictions
- **TX, NY, PA**: Decline loans with rates > 20.99% for 36-month terms
- **TX, NY, PA**: Decline loans with rates > 23.99% for 60-month terms
- **PA only**: Allow 60-month terms up to 25.99% (special exception)

### Monotonic Rate Requirement
- Interest rates must be **monotonic** (non-decreasing) with loan amount
- Within each `app_id` and `loan_term` combination
- Example: If 5,000 loan = 6.5%, then 10,000 loan ≥ 6.5%

##  Your Mission
Load the business rules dataset and identify **all violations** of these rules - make sure you test all relevant cases.


In [6]:
# Load business rules validation dataset
business_data = pd.read_parquet('business_rules_validation.parquet')

print("BUSINESS RULES DATASET LOADED")
print(f"Dataset shape: {business_data.shape}")

print("Dataset columns:")
print(business_data.columns.tolist())

print("Sample data:")
business_data.head(10)

BUSINESS RULES DATASET LOADED
Dataset shape: (570, 8)
\Dataset columns:
['app_id', 'loan_amount', 'loan_term', 'requested_amount', 'state', 'is_approved', 'int_rate', 'credit_score']
Sample data:


,app_id,loan_amount,loan_term,requested_amount,state,is_approved,int_rate,credit_score
0,2000,5000,36,27500,CO,True,13.99,702
1,2000,8000,36,27500,CO,True,14.99,702
2,2000,10000,36,27500,CO,True,15.99,702
3,2000,15000,36,27500,CO,True,16.99,702
4,2000,20000,36,27500,CO,True,17.99,702
5,2000,25000,36,27500,CO,True,18.99,702
6,2000,30000,36,27500,CO,True,19.99,702
7,2000,5000,60,27500,CO,True,14.99,702
8,2000,8000,60,27500,CO,True,15.99,702
9,2000,10000,60,27500,CO,True,16.99,702


# Your Findings & Recommendations

## Summary of Investigation

**Use this space to document your findings:**

### Initial assignment Dataset Analysis:



### Second assignment Dataset Analysis:


